In [5]:
import medmnist
from medmnist import INFO

In [8]:
data_flag = "dermamnist"
download = True

NUM_EPOCHS = 3
BATCH_SIZE = 128
lr = 0.001

info = INFO[data_flag]
task = info['task']
num_channels = info['n_channels']
num_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

print(DataClass)

task: multi-class
n_channels: 3
n_classes: 7
